# AML SQL Analysis Notebook

Mini–Proyecto de análisis de patrones de fraude bancario usando SQL.

Este notebook incluye 8 consultas SQL que detectan:
- cash structuring / smurfing
- high-risk jurisdictions
- rapid movement
- repetitive destination patterns
- large transactions
- velocity rules
- cumulative 24h movement
- high-risk customer + high-risk destination

---

In [1]:
%%sql
USE aml_system;

## 1. Cash Structuring / Smurfing
Tres o más transacciones repetidas por debajo de 500 USD.

In [2]:
%%sql
SELECT 
    T.cliente_id,
    C.nombre_cliente,
    COUNT(*) AS num_transacciones,
    SUM(T.monto_usd) AS total_movido
FROM Transacciones T
INNER JOIN Clientes C ON C.id_cliente = T.cliente_id
WHERE T.monto_usd < 500
GROUP BY T.cliente_id, C.nombre_cliente
HAVING COUNT(*) >= 3;

## 2. High Risk Jurisdictions
Transacciones enviadas a jurisdicciones de alto riesgo.

In [3]:
%%sql
SELECT 
    T.*, 
    C.nombre_cliente,
    C.nivel_riesgo
FROM Transacciones T
INNER JOIN Clientes C ON C.id_cliente = T.cliente_id
WHERE T.pais_destino IN ('UAE', 'Panama', 'Cayman Islands');

## 3. Rapid Movement (<5 minutos entre transacciones)
Dos transacciones del mismo cliente en menos de 5 minutos.

In [4]:
%%sql
SELECT 
    T1.cliente_id,
    C.nombre_cliente,
    T1.id_transaccion AS t1,
    T2.id_transaccion AS t2,
    TIMESTAMPDIFF(MINUTE, T1.timestamp_transaccion, T2.timestamp_transaccion) AS minutos_diff
FROM Transacciones T1
INNER JOIN Transacciones T2 
    ON T1.cliente_id = T2.cliente_id
    AND T2.timestamp_transaccion > T1.timestamp_transaccion
INNER JOIN Clientes C ON C.id_cliente = T1.cliente_id
WHERE TIMESTAMPDIFF(MINUTE, T1.timestamp_transaccion, T2.timestamp_transaccion) <= 5;

## 4. Large Transactions (>=10,000 USD)
Transacciones grandes que rebasan umbrales regulatorios.

In [5]:
%%sql
SELECT 
    T.*, 
    C.nombre_cliente
FROM Transacciones T
INNER JOIN Clientes C ON C.id_cliente = T.cliente_id
WHERE T.monto_usd >= 10000;

## 5. Repetitive Destination Pattern
Tres o más envíos al mismo país por el mismo cliente.

In [6]:
%%sql
SELECT 
    cliente_id,
    pais_destino,
    COUNT(*) AS veces
FROM Transacciones
GROUP BY cliente_id, pais_destino
HAVING COUNT(*) >= 3;

## 6. High Risk Customer + High Risk Destination
Clientes de alto riesgo enviando dinero a países de alto riesgo.

In [7]:
%%sql
SELECT 
    T.*, 
    C.nombre_cliente,
    C.nivel_riesgo
FROM Transacciones T
INNER JOIN Clientes C ON C.id_cliente = T.cliente_id
WHERE C.nivel_riesgo = 'HIGH'
  AND T.pais_destino = 'Panama';

## 7. Velocity Rule (>5 transacciones por día)
Regla de velocidad: actividad anómala dentro del mismo día.

In [8]:
%%sql
SELECT 
    cliente_id,
    DATE(timestamp_transaccion) AS fecha,
    COUNT(*) AS transacciones
FROM Transacciones
GROUP BY cliente_id, DATE(timestamp_transaccion)
HAVING transacciones >= 5;

## 8. Cumulative Amount > 10,000 USD en 24 horas
Suma acumulada móvil de 24h por cliente.

In [9]:
%%sql
SELECT
    T1.cliente_id,
    C.nombre_cliente,
    SUM(T2.monto_usd) AS total_24h
FROM Transacciones T1
INNER JOIN Transacciones T2 
    ON T1.cliente_id = T2.cliente_id
    AND T2.timestamp_transaccion BETWEEN T1.timestamp_transaccion 
    AND DATE_ADD(T1.timestamp_transaccion, INTERVAL 1 DAY)
INNER JOIN Clientes C ON C.id_cliente = T1.cliente_id
GROUP BY T1.cliente_id, C.nombre_cliente
HAVING total_24h > 10000;